In [3]:
## PROVA SENZA OVERLAP con rimozione background
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
import warnings
import gdown
import os
import shutil
from matplotlib.backends.backend_pdf import PdfPages
from google.colab import drive

# ignoro gli avvisi di runtime
warnings.filterwarnings("ignore", category=RuntimeWarning)

# scarico i file necessari dal repository GitHub
!wget -O "Lettura_Dati.py" https://raw.githubusercontent.com/microlab-unibg/Sense2Gol-Pulse-vital-Parameters/Caricamento-file-e-collegamento-al-drive/MetodiDiRimozione/Lettura_Dati.py
!wget -O "Background.npy" https://raw.githubusercontent.com/microlab-unibg/Sense2Gol-Pulse-vital-Parameters/Caricamento-file-e-collegamento-al-drive/MetodiDiRimozione/Background.npy

# importo la funzione di lettura dati
import sys
sys.path.insert(0, './')
from Lettura_Dati import leggi_file

# carico il background dal file .npy scaricato
background = np.load("Background.npy")
print("Dimensione del background:", background.size)

# carico il file di dati da Google Drive
file_url = "https://drive.google.com/uc?id=15LSYaKYgg9GrVJgwSE_dBTPDefqShAaD"
gdown.download(file_url, 'dati.txt', quiet=False)

# leggo il file
file_path = "dati.txt"
frames_array = leggi_file(file_path)

# parametri per FFT e spettrogramma
V = 255
spectrogram = np.zeros((len(frames_array), V // 2), dtype=float)

# calcolo lo spettrogramma
for i, frame in enumerate(frames_array):
    yf_high1 = fft(frame[0:V], axis=0)
    spectrogram[i, :] = np.abs(yf_high1[:V // 2])

# calcolo il tempo totale e le frequenze
T_frame = 1
tempo_totale = len(frames_array) * T_frame
xf_high = fftfreq(V, T_frame / V)[:V // 2]

# rimuovo il background
spectrogram_without_baseline = spectrogram - background[:V // 2]
spectrogram_without_baseline_real = np.real(spectrogram_without_baseline)

# frequenze tra 0 e 30 Hz
freq_min = 0
freq_max = 30
idx_freq_min = np.argmax(xf_high >= freq_min)
idx_freq_max = np.argmax(xf_high > freq_max)

# parametri per l'overlap
Nm = 9  # Numero di frame per blocco (senza sovrapposizione)
i = 0

# creo un PDF per salvare i grafici
pdf_filename = "grafici_outputM2_senza_overlap.pdf"
with PdfPages(pdf_filename) as pdf:
    # Numero totale di finestre
    while True:
        # indice di inizio e fine finestra
        start_idx = i * Nm
        end_idx = start_idx + Nm

        # Verifico che ci siano abbastanza dati
        block_frames = spectrogram_without_baseline_real[start_idx:end_idx]
        if len(block_frames) < Nm:
            print('Non ci sono abbastanza valori')
            break
        else:
            # Calcolo la media del blocco
            mean_fft_block = np.mean(block_frames, axis=0)
            mean_fft_block_real = np.real(mean_fft_block[idx_freq_min:idx_freq_max])

            # Frequenze selezionate tra 0 e 30 Hz
            xf_selected = xf_high[idx_freq_min:idx_freq_max]

            # Plot della finestra corrente
            plt.figure(figsize=(6, 3))
            plt.plot(xf_selected, mean_fft_block_real)
            plt.title(f'Finestra {i+1}, con valori compresi tra {start_idx} e {end_idx}', fontsize=16)
            plt.xlabel('Frequenza [Hz]', fontsize=16)
            plt.ylabel('Ampiezza [au]', fontsize=16)
            plt.grid(True)

            # Salvo il grafico nel PDF
            pdf.savefig()
            plt.close()

            i += 1

# carico il PDF su google drive
drive.mount('/content/drive', force_remount=True)
percorso_su_drive = '/content/drive/Shareddrives/Radar/Vital parameter monitoring/Sense2GoL_pulse_data/FileOutput/'
os.makedirs(percorso_su_drive, exist_ok=True)
shutil.move(pdf_filename, os.path.join(percorso_su_drive, pdf_filename))
print("File caricati con successo su Google Drive nel drive condiviso Radar!")



--2024-11-09 10:27:49--  https://raw.githubusercontent.com/microlab-unibg/Sense2Gol-Pulse-vital-Parameters/Caricamento-file-e-collegamento-al-drive/MetodiDiRimozione/Lettura_Dati.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2371 (2.3K) [text/plain]
Saving to: ‘Lettura_Dati.py’

Lettura_Dati.py     100%[===================>]   2.32K  --.-KB/s    in 0s      

2024-11-09 10:27:49 (34.3 MB/s) - ‘Lettura_Dati.py’ saved [2371/2371]

--2024-11-09 10:27:49--  https://raw.githubusercontent.com/microlab-unibg/Sense2Gol-Pulse-vital-Parameters/Caricamento-file-e-collegamento-al-drive/MetodiDiRimozione/Background.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubuser

Downloading...
From: https://drive.google.com/uc?id=15LSYaKYgg9GrVJgwSE_dBTPDefqShAaD
To: /content/dati.txt
100%|██████████| 771k/771k [00:00<00:00, 41.5MB/s]


Non ci sono abbastanza valori
Mounted at /content/drive
File caricati con successo su Google Drive nel drive condiviso Radar!


In [4]:
##PROVA CON OVERLAP PER FAR ARRIVARE LE FREQUENZE A 120 E POI VANNO FILTRATE FINO A 30
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
import warnings
import gdown
import os
import shutil
from matplotlib.backends.backend_pdf import PdfPages
from google.colab import drive

# Ignora gli avvisi di runtime
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Scarica i file necessari dal repository GitHub
!wget -O "Lettura_Dati.py" https://raw.githubusercontent.com/microlab-unibg/Sense2Gol-Pulse-vital-Parameters/Caricamento-file-e-collegamento-al-drive/MetodiDiRimozione/Lettura_Dati.py
!wget -O "Background.npy" https://raw.githubusercontent.com/microlab-unibg/Sense2Gol-Pulse-vital-Parameters/Caricamento-file-e-collegamento-al-drive/MetodiDiRimozione/Background.npy

# Importa la funzione di lettura dati
import sys
sys.path.insert(0, './')
from Lettura_Dati import leggi_file

# Carica il background dal file .npy scaricato
background = np.load("Background.npy")
print("Dimensione del background:", background.size)

# Scarica il file di dati da Google Drive
file_url = "https://drive.google.com/uc?id=15LSYaKYgg9GrVJgwSE_dBTPDefqShAaD"
gdown.download(file_url, 'dati.txt', quiet=False)

# Leggi il file
file_path = "dati.txt"
frames_array = leggi_file(file_path)

# Parametri per FFT e spettrogramma
V = 256
spectrogram = np.zeros((len(frames_array), V // 2), dtype=float)

# Calcola lo spettrogramma
for i, frame in enumerate(frames_array):
    yf_high1 = fft(frame[0:V], axis=0)
    spectrogram[i, :] = np.abs(yf_high1[:V // 2])

# Calcola le frequenze
T_frame = 1
xf_high = fftfreq(V, T_frame / V)[:V // 2]

# Rimozione del background
spectrogram_without_baseline = spectrogram - background[:V // 2]
spectrogram_without_baseline_real = np.real(spectrogram_without_baseline)

# Frequenze tra 0 e 30 Hz
freq_min = 0
freq_max = 30
idx_freq_min = np.argmax(xf_high >= freq_min)
idx_freq_max = np.argmax(xf_high > freq_max)
xf_selected = xf_high[idx_freq_min:idx_freq_max]

# Parametri per la finestratura con sovrapposizione
Nm = 9
overlap = 4
i = 0

# Crea un PDF per salvare i grafici
pdf_filename = "grafici_outputM2_con_overlap.pdf"
with PdfPages(pdf_filename) as pdf:
    # Ciclo per processare ogni finestra con sovrapposizione
    while True:
        # Indici di inizio e fine finestra
        start_idx = i * (Nm - overlap)
        end_idx = start_idx + Nm

        # Verifica che ci siano abbastanza dati
        block_frames = spectrogram_without_baseline_real[start_idx:end_idx]
        if len(block_frames) < Nm:
            print('Non ci sono abbastanza valori per la finestra')
            break
        else:
            # Calcola la media del blocco
            mean_fft_block = np.mean(block_frames, axis=0)
            mean_fft_block_real = np.real(mean_fft_block[idx_freq_min:idx_freq_max])

            # Plot della finestra corrente
            plt.figure(figsize=(6, 3))
            plt.plot(xf_selected, mean_fft_block_real)
            plt.title(f'Finestra {i+1}, con valori compresi tra {start_idx} e {end_idx}', fontsize=16)
            plt.xlabel('Frequenza [Hz]', fontsize=16)
            plt.ylabel('Ampiezza [au]', fontsize=16)
            plt.grid(True)

            # Salva il grafico nel PDF
            pdf.savefig()
            plt.close()

            i += 1

# carico il PDF
drive.mount('/content/drive', force_remount=True)
percorso_su_drive = '/content/drive/Shareddrives/Radar/Vital parameter monitoring/Sense2GoL_pulse_data/FileOutput/'
os.makedirs(percorso_su_drive, exist_ok=True)
shutil.move(pdf_filename, os.path.join(percorso_su_drive, pdf_filename))
print("File caricati con successo su Google Drive nel drive condiviso Radar!")



--2024-11-09 10:32:28--  https://raw.githubusercontent.com/microlab-unibg/Sense2Gol-Pulse-vital-Parameters/Caricamento-file-e-collegamento-al-drive/MetodiDiRimozione/Lettura_Dati.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2371 (2.3K) [text/plain]
Saving to: ‘Lettura_Dati.py’

Lettura_Dati.py     100%[===================>]   2.32K  --.-KB/s    in 0s      

2024-11-09 10:32:28 (31.0 MB/s) - ‘Lettura_Dati.py’ saved [2371/2371]

--2024-11-09 10:32:28--  https://raw.githubusercontent.com/microlab-unibg/Sense2Gol-Pulse-vital-Parameters/Caricamento-file-e-collegamento-al-drive/MetodiDiRimozione/Background.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubuser

Downloading...
From: https://drive.google.com/uc?id=15LSYaKYgg9GrVJgwSE_dBTPDefqShAaD
To: /content/dati.txt
100%|██████████| 771k/771k [00:00<00:00, 37.7MB/s]


Non ci sono abbastanza valori per la finestra
Mounted at /content/drive
File caricati con successo su Google Drive nel drive condiviso Radar!
